In [1]:
%pip install gensim
%pip install torch
%pip install Pillow
%pip install requests
%pip install open_clip_torch
%pip install torchvision


  Using cached gensim-4.3.3-cp39-cp39-macosx_11_0_arm64.whl.metadata (8.3 kB)
  Using cached numpy-1.26.4-cp39-cp39-macosx_11_0_arm64.whl.metadata (61 kB)
  Using cached scipy-1.13.1-cp39-cp39-macosx_12_0_arm64.whl.metadata (60 kB)
  Using cached smart_open-7.1.0-py3-none-any.whl.metadata (24 kB)
  Using cached wrapt-1.17.2-cp39-cp39-macosx_11_0_arm64.whl.metadata (6.4 kB)
Using cached gensim-4.3.3-cp39-cp39-macosx_11_0_arm64.whl (24.0 MB)
Using cached numpy-1.26.4-cp39-cp39-macosx_11_0_arm64.whl (14.0 MB)
Using cached scipy-1.13.1-cp39-cp39-macosx_12_0_arm64.whl (30.3 MB)
Using cached smart_open-7.1.0-py3-none-any.whl (61 kB)
Using cached wrapt-1.17.2-cp39-cp39-macosx_11_0_arm64.whl (38 kB)
Note: you may need to restart the kernel to use updated packages.
  Using cached torch-2.6.0-cp39-none-macosx_11_0_arm64.whl.metadata (28 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached networkx-3.2.1-py3-none-any.whl.metadata (5.2 kB)
  Using cached jinja2-3.1.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import gensim.downloader as api # pip install gensim
from txt2png import txt_to_png
if torch.backends.mps.is_available():
    from open_clip_test_mps import clip_loss, device, preprocess
elif torch.cuda.is_available():
    from open_clip_test_cuda import clip_loss, device, preprocess

In [19]:
word_model_name = "glove-twitter-25"
word_model = api.load(word_model_name)
embedding_dim = 25
x_dim = 25
y_dim = 8
output_size = y_dim * x_dim
ascii_chars = " .,:;+*#@$%&0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ-_=|/\\()[]{}"
num_chars = len(ascii_chars)
char_to_index = {char: i for i, char in enumerate(ascii_chars)}
index_to_char = {i: char for i, char in enumerate(ascii_chars)}

class ASCIIArtGenerator(nn.Module):
    def __init__(self, input_size, output_size, num_chars):
        super(ASCIIArtGenerator, self).__init__()
        self.fc1 = nn.Linear(input_size, 512)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, output_size * num_chars)
        self.num_chars = num_chars
        self.output_size = output_size

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x.view(-1, self.output_size, self.num_chars)

def generate_ascii_art(model, embedding, temperature=1.0):
    model.eval()
    with torch.no_grad():
        output = model(embedding.unsqueeze(0))  # Shape: [1, output_size, num_chars]
        
        # Apply temperature scaling to logits
        scaled_logits = output / temperature
        
        # Convert to probabilities
        probs = F.softmax(scaled_logits, dim=2)
        
        # Sample from the distribution
        # We'll use multinomial sampling which selects based on probabilities
        indices = torch.multinomial(probs.view(-1, num_chars), 1).squeeze(-1)
        
        ascii_grid = ""
        for i in range(y_dim):
            for j in range(x_dim):
                ascii_grid += index_to_char[indices[i * x_dim + j].item()]
            ascii_grid += "\n"
        return ascii_grid

model = ASCIIArtGenerator(embedding_dim, output_size, num_chars)

In [22]:
embedding = torch.tensor(word_model["cat"])
optimizer = optim.Adam(model.parameters(), lr = .001)
for i in range(10):
    ascii_art = generate_ascii_art(model, embedding, temperature=.5)
    ascii_png = txt_to_png(ascii_art, output_png="temp.png", save=True)
    image_tensor = preprocess(ascii_png).unsqueeze(0).to(device, dtype=torch.float32)
    optimizer.zero_grad()
    candidate_labels = ["bird", "dog", "cat", "castle"]
    true_label = 2
    loss = clip_loss(image_tensor, candidate_labels, true_label)
    loss.backward()
    optimizer.step()
    print(f"Iteration {i}, Loss: {loss.item()}")
    print(ascii_art)
    #print()

Iteration 0, Loss: 2.1197566986083984
O:(T=ogcj)=hW*.c3-qd9lIA]
c).0yY8f@hvDJR_y}k=0Mf@.t
8s9}IG6H)Vmm9r|5E]xnYx{pN
VQmmc]nIpcBQ9GOq\+5(_3rx|
P@v=ws3 rSbTXhSM0bCx:70D8
 U3TzeaOQ=*y5N9|x)4PzsQel
scASw5yShkR]B*;_=U$r8w)-+
Jn&4wNr#q KC\*6NbJZUiC93,

Iteration 1, Loss: 2.2376253604888916
}uch9ilOhHSiwbLv*7;6rjB%E
p5h]S}%Nuf{.O|xLIwrF1@uZ*
h3R07BEY2zU(RD4_uojgx2 Oo
{tA6pR4Qc9okCT6(+@eL8hKx;
$%khdcKoOt:d%Dws\NpRj$lw;
gz9Qt9eoS)(%kFyhmE[]4p\o_
(R HA\5(Fo{Ued5[n_S;8\E7b
F*a37lKTMOZVZkt%ul7}P3G#A

Iteration 2, Loss: 0.708647608757019
=PEXnIMRq2p:yc2fdf27cQ2,Q
YS9pTQ91+APr_lhWo44$dave(
G/WPy$.OdZtzP)aouf2LqG_k4
s-{Qf4._${5ro(0I)_rS2)7Nr
U4VJPa([JYOP+aEtPHGmdERnf
c;vm5%Af#308y#GcWLDUPJYR5
&:e)$WcfBHvc[[xyrtmiUK7kL
*nlNuvDuS;99Q3PUmCJ70[(k\

Iteration 3, Loss: 1.9367587566375732
p2{.G@;EcwS6,8dp)c+Og+H+2
Oh43ibl7DUmw;0oGWMAJ:r7.s
hkLN:e&HNxB[/#F#gZ8 C5/Q6
\[NfDMwACKPwv|A[|ks)-;:vy
gG{TUxaE]I;QwwIdG6/X@hd  
+0OAG@SQq*FxVDw.dQZ7e:2Ve
{Iiu,jqnV,;T4jk/W/ja}*acz
#hr\[3nkWsjhbnmJFW8|67ce-

Iteration 4, 